## Import the necessary functions and libraries

In [1]:
from BoW import TF_IDF
from Preprocessing import getDataFrameFromData
from rnn import RNN
from networkFunctions import train
from ffnn import FFNN
import torch 

## Read Data

In [5]:
dataFrame, labels = getDataFrameFromData('Archive/arxiv_train.csv')
dataFrame

,id,abstract,label
0,31716,automatic meeting analysis is an essential f...,eess
1,89533,we propose a protocol to encode classical bi...,quantph
2,82700,a number of physically intuitive results for...,quantph
3,78830,in the last decade rareearth hexaborides hav...,physics
4,94948,we introduce the weak barycenter of a family...,stat
...,...,...,...
95,36235,in this paper a new framework for optimizing...,eess
96,38205,reconfigurable intelligent surface ris techn...,eess
97,55889,we explore emphfolded spinning string config...,hepth
98,32525,in this paper we formulate a simple algorith...,eess


## Calculate word frequency and TF_IDF scores

In [7]:
torchTensor  = TF_IDF(dataFrame, 'abstract', labels)
torchTensor

c:\UIS\DAT550\dat550-prosjekt\BoW.py:26: UserWarning: torch.sparse.SparseTensor(indices, values, shape, *, device=) is deprecated.  Please use torch.sparse_coo_tensor(indices, values, shape, dtype=, device=). (Triggered internally at ..\torch\csrc\utils\tensor_new.cpp:620.)
  tensor = torch.sparse.FloatTensor(i, v, torch.Size(shape))


tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

## Initalize parameters for the networks

In [ ]:
inputSize = torchTensor.shape[1]
hiddenSize = 100
numLayers = 1
PATH = "Models/"

## Create Models

In [ ]:
RNNmodel = RNN(inputSize, hiddenSize, numLayers)
FFNNmodel = FFNN(torchTensor.shape[1])

## Load state dict for models
#### If there are no saved model states, then a new one will be trained from scratch

In [ ]:
try:
	RNNmodel.load_state_dict(torch.load(PATH + "RNNmodel.pt"))
	FFNNmodel.load_state_dict(torch.load(PATH + "FFNNmodel.pt"))
	print("Loaded model states")
except FileNotFoundError:
	print("File not found, initializing training...\n")
	print("Training RNN...\n")
	modelR = train(RNNmodel, "adam", torchTensor, labels)
	print("Training FFNN...\n")
	modelFF = train(FFNNmodel, "adam", torchTensor, labels)

## Save model state dict of the neural networks

In [ ]:
torch.save(RNNmodel.state_dict(), PATH + "RNNmodel.pt")
torch.save(FFNNmodel.state_dict(), PATH + "FFNNmodel.pt")